<a href="https://colab.research.google.com/github/AirPro/HandsOnMod3Freidrd/blob/main/HandsOnMod3Freidrd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 3 Hands On Assignment By: Robert Freid
It is in the hands-on. You need to watch the section 5 in the LinkedIn course which link is provided in the hands-on section. So, section 5 has different subtopic video list, you need to repeat exactly what has been done in these videos but using Google Colab.
The title of these subsections:

* Using a pre-trained network for object recognition
* Extracting features with a pre-trained neural network
* Training a new neural network with extracted features
* Making predictions with transfer learning


In [1]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
from keras.applications import vgg16
import tensorflow as tf
print(tf.__version__)


2.11.0


## Using A Pre-Trained Network for Object Recognition
Repreduce the exercise from the LinkedIn video.

In [2]:
# Load the Keras VGG16 - Model is pretrained against the ImageNet database
model = vgg16.VGG16()

In [3]:
# Upload the file from local computer: bay.jpg
from google.colab import files
uploaded = files.upload()

Saving bay.jpg to bay.jpg


In [5]:
# Load the Image file "bay.jpg" and resize it to 224 x 224 pixels
img = tf.keras.utils.load_img("bay.jpg", target_size=(224, 224))
# Convert image to numpy array
x = tf.keras.utils.img_to_array(img)

In [6]:
# Add a fourth dimension due to Karas expecting a list of images for consistency
x = np.expand_dims(x, axis=0)

In [7]:
# Normalize imput image values to range used for training nthe neural network
x = vgg16.preprocess_input(x)

In [8]:
# Initaite the convolutional neural network to make a prediction
predictions = model.predict(x)

1/1 [==============================] - 2s 2s/step


In [9]:
# Check for hthe names of the predicted classes. Index 0 is the results for the first image.
predicted_classes = vgg16.decode_predictions(predictions)

35363/35363 [==============================] - 0s 0us/step


In [10]:
# Print out the predictions for our test image
print("Top predictions for this image: ")
for imagenet_id, name, likelihood in predicted_classes[0]:
  print("Prediction: {} - {:2f}".format(name, likelihood) )

Top predictions for this image: 
Prediction: seashore - 0.395213
Prediction: promontory - 0.326129
Prediction: lakeside - 0.119613
Prediction: breakwater - 0.062801
Prediction: sandbar - 0.045267
